# 생산계획 최적화 따라하기

- 출처 : 중소벤처기업부, Korea AI Manufacturing Platform(KAMP), 생산계획 최적화 AI 데이터셋, 인공지능 중소벤처 제조 플랫폼, 2021.12.27
         https://www.kamp-ai.kr/front/dataset/AiDataDetail.jsp?AI_SEARCH=&page=2&DATASET_SEQ=30&EQUIP_SEL=&FILE_TYPE_SEL=&GUBUN_SEL=&WDATE_SEL=


- 필요데이터 : machine_info.csv, order_info.csv       

## 사용 알고리즘

### Genetic Algorithm
- 유전 알고리즘
- 변이과정과 적자생존으로 최적의 개체를 유전하는 자연현상을 최적화에 적용한 알고리즘. 임의의 조합으로 집합(population) 생성, 최적도(fitness)가 높은 조합해를 선택(selection), 재조합(crossover), 변이(mutation)하는 일련의 과정(generation)을 반복, 더 나은 조합해를 찾는 과정

### 시사점
- NC설비 공정의 작업 우선순위 설정 개선 및 납기지연 감소(20% 이상) 기대
- 생산 계획을 최적화하는 알고리즘 개발
- 다양한 제조공정에 적용 기대

선박용 볼트 및 너트 제조공정 -> 선박 엔진에 들어가는 볼트,너트는 장난이 아니에요~~~

#### 데이터 정의

- 독립변수
    - 영업납기(time)
    - 중산도면(item), NC설비번호(machine)
    - 단가(cost)
    - 선급(prepaid), 긴급(urgent)
    - 수량(qty), 설비일일용량(CYCLETIME, capacity)
- 종속변수
    - 실제 생산량(amount of item), 패널티(penalty ??)

#### 분석 단계별 프로세스
1. 라이브러리/ 데이터 로드
2. 데이터 유효성 검증
3. 종류 및 개수 확인
4. 데이터 정제
5. 변수 생성
6. 유전 알고리즘 구축 (세부함수, 파라미터 할당)
7. 알고리즘 실행
8. 결과분석 및 해석

#### 1 라이브러리/ 데이터 로드

In [2]:
import random as rnd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

from numpy.random import randint, rand
from tqdm import tqdm

In [3]:
machine_info = pd.read_csv('./data/machine_info.csv')
order_info = pd.read_csv('./data/order_info.csv')

In [5]:
machine_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21151 entries, 0 to 21150
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   item      21151 non-null  object 
 1   machine   21149 non-null  float64
 2   capacity  21151 non-null  float64
dtypes: float64(2), object(1)
memory usage: 495.9+ KB


In [6]:
order_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   영업납기    127 non-null    object
 1   중산도면    127 non-null    object
 2   단가      127 non-null    int64 
 3   수량      127 non-null    int64 
 4   선급      46 non-null     object
dtypes: int64(2), object(3)
memory usage: 5.1+ KB


##### 열이름 변경

In [10]:
order_info = order_info.rename(columns={
    '영업납기':'time',
    '중산도면':'item',
    '단가':'cost',
    '수량':'qty',
    '선급':'urgent'
})
# 왜 선급인데 prepaid가 아니라 urgent야?

In [13]:
order_info.tail()

,time,item,cost,qty,urgent
122,2021-08-15,Z00807,10450,300,NaN
123,2021-08-18,069559,242250,28,검사품
124,2021-08-19,K00614,6750,12,NaN
125,2021-08-31,K04037,7810,400,검사품
126,2021-09-15,Z00807,10450,325,NaN
